In [82]:
import pandas as pd
df = pd.read_csv('~/Downloads/Cohort 3 Data - Mentees.csv')
#Clean DF
df_flitered = df.filter(items=["Id", 
                 "What's your email?",
                 'What role are you looking for help in?',
                 "Which best describes the position you're in?",
                 'How many years of professional experience do you have in your current role?',
                 'What is the main goal you want help with from your mentor?',
                 'How many years of experience do you have as a founder?',
                 'What industry are you looking for help with?',
                 'What company stage do you want your mentor to be in?',
                 'What timezone are you in?',
                 'Have you been matched before?',
                 "Tell us why you want to be a part of Lenny's Mentorship Program!",
                 'What is the main goal you want help with from your mentor?'
                ])

print(df_flitered)

        Id  What's your email? What role are you looking for help in?  \
0      1.0                 NaN                         Career Pathing   
1      2.0                 NaN                                Product   
2      3.0                 NaN                                Founder   
3      4.0                 NaN                                Product   
4      5.0                 NaN                                Product   
..     ...                 ...                                    ...   
236  237.0                 NaN                                Product   
237  238.0                 NaN                                Product   
238  239.0                 NaN                                Product   
239  240.0                 NaN                                Product   
240    NaN                 NaN                                    NaN   

    Which best describes the position you're in?  \
0                                            NaN   
1                  

In [83]:
print(df_flitered.columns.values)

['Id' "What's your email?" 'What role are you looking for help in?'
 "Which best describes the position you're in?"
 'How many years of professional experience do you have in your current role?'
 'What is the main goal you want help with from your mentor?'
 'How many years of experience do you have as a founder?'
 'What industry are you looking for help with?'
 'What company stage do you want your mentor to be in?'
 'What timezone are you in?' 'Have you been matched before?'
 "Tell us why you want to be a part of Lenny's Mentorship Program!"
 'What is the main goal you want help with from your mentor?']


In [84]:
def clean_multiselect(x):
    if isinstance(x, str):
        return list(map(str.strip,x.split(',')))
    else:
        return []


In [85]:
test = df_flitered['What industry are you looking for help with?']
test_list = test.to_list()
nested_list = list(map(clean_multiselect,test_list))
print(test_list)
print(nested_list)
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()



[nan, 'Health & Wellness', 'DevOps / Dev tools, B2B Saas', 'B2B Saas, Fintech', 'B2B Saas', 'Marketplace, Fintech', 'Fintech', 'Media, B2B Saas', 'Cybersecurity, B2B Saas', 'Consumer, Media', 'HR, Marketplace', 'DevOps / Dev tools', 'Media, E-commerce', 'B2B Saas', 'Food', 'Health & Wellness, Community', 'B2B Saas, Consumer', 'Consumer', 'Marketplace, Fintech', 'Marketplace, Travel', 'Future of work, B2B Saas', 'E-commerce, Marketplace', 'DevOps / Dev tools, Future of work', 'B2B Saas, Consumer', 'B2B Saas, Cybersecurity', 'Future of work, B2B Saas', 'Health & Wellness, Consumer', 'B2B Saas', 'B2B Saas, AI', 'B2B Saas, Construction Tech', 'B2B Saas', 'IaaS/Cloud computing', 'E-commerce, Climate Tech', 'Fintech', 'B2B Saas, Consumer', 'Fintech', 'Health Tech', nan, 'Fintech', nan, 'Supply Chain, IoT', 'Marketplace, HR', 'Fintech, Marketplace', 'Fintech, Consumer', 'MarTech, EdTech', 'Consumer, B2B Saas', 'Consumer', 'B2B Saas, Consumer', 'E-commerce, Consumer', 'Marketplace, Consumer', 

In [86]:
t = [['hotdog','soda'],['car','hotdog']]
t = [['a'], ['Health & Wellness'], ['DevOps / Dev tools', ' B2B Saas'], ['B2B Saas', ' Fintech'], ['B2B Saas']]
t = nested_list
print(t)
print(mlb.fit_transform(t))
print(mlb.classes_)

[[], ['Health & Wellness'], ['DevOps / Dev tools', 'B2B Saas'], ['B2B Saas', 'Fintech'], ['B2B Saas'], ['Marketplace', 'Fintech'], ['Fintech'], ['Media', 'B2B Saas'], ['Cybersecurity', 'B2B Saas'], ['Consumer', 'Media'], ['HR', 'Marketplace'], ['DevOps / Dev tools'], ['Media', 'E-commerce'], ['B2B Saas'], ['Food'], ['Health & Wellness', 'Community'], ['B2B Saas', 'Consumer'], ['Consumer'], ['Marketplace', 'Fintech'], ['Marketplace', 'Travel'], ['Future of work', 'B2B Saas'], ['E-commerce', 'Marketplace'], ['DevOps / Dev tools', 'Future of work'], ['B2B Saas', 'Consumer'], ['B2B Saas', 'Cybersecurity'], ['Future of work', 'B2B Saas'], ['Health & Wellness', 'Consumer'], ['B2B Saas'], ['B2B Saas', 'AI'], ['B2B Saas', 'Construction Tech'], ['B2B Saas'], ['IaaS/Cloud computing'], ['E-commerce', 'Climate Tech'], ['Fintech'], ['B2B Saas', 'Consumer'], ['Fintech'], ['Health Tech'], [], ['Fintech'], [], ['Supply Chain', 'IoT'], ['Marketplace', 'HR'], ['Fintech', 'Marketplace'], ['Fintech', 'Con

In [87]:
mlb_df[mlb.classes_] = mlb.fit_transform(t)
bigger = pd.concat([df_flitered,mlb_df],axis=1)
print(mlb_df)
display(bigger)

     AI  Agency  AgriTech  Agritech  B2B Saas  Biotech  Climate Tech  \
0     0       0         0         0         0        0             0   
1     0       0         0         0         0        0             0   
2     0       0         0         0         1        0             0   
3     0       0         0         0         1        0             0   
4     0       0         0         0         1        0             0   
..   ..     ...       ...       ...       ...      ...           ...   
236   1       0         0         0         0        0             0   
237   0       0         0         0         1        0             0   
238   0       0         0         0         1        0             0   
239   1       0         0         0         0        0             0   
240   0       0         0         0         0        0             0   

     Community  Construction Tech  Consumer  ...  \
0            0                  0         0  ...   
1            0                 

,Id,What's your email?,What role are you looking for help in?,Which best describes the position you're in?,How many years of professional experience do you have in your current role?,What is the main goal you want help with from your mentor?,How many years of experience do you have as a founder?,What industry are you looking for help with?,What company stage do you want your mentor to be in?,What timezone are you in?,...,Medtech selling to insurance with d2c marketing,Non-profit,PropTech,Proptech,Restaurant Technology,Smart Home,Supply Chain,Travel,Web3,igaming affiliation
0,1.0,NaN,Career Pathing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2.0,NaN,Product,Individual contributor (IC),6-8,Level up in career,NaN,Health & Wellness,Series A,Central European Time (GMT +2),...,0,0,0,0,0,0,0,0,0,0
2,3.0,NaN,Founder,NaN,NaN,NaN,0-2,"DevOps / Dev tools, B2B Saas",Seed or earlier,Eastern Time (GMT -4),...,0,0,0,0,0,0,0,0,0,0
3,4.0,NaN,Product,Individual contributor (IC),0-2,Transition into product management,NaN,"B2B Saas, Fintech",Series D+,Eastern Time (GMT -4),...,0,0,0,0,0,0,0,0,0,0
4,5.0,NaN,Product,Manager of ICs,0-2,Transition into product management,NaN,B2B Saas,Series B,Central European Time (GMT +2),...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,237.0,NaN,Product,Individual contributor (IC),0-2,Become better in my IC role,NaN,"Cybersecurity, AI",Public company,Pacific Time (GMT -7),...,0,0,0,0,0,0,0,0,0,0
237,238.0,NaN,Product,Individual contributor (IC),0-2,Level up in career,NaN,"Health & Wellness, B2B Saas",Series C,Pacific Time (GMT -7),...,0,0,0,0,0,0,0,0,0,0
238,239.0,NaN,Product,Individual contributor (IC),0-2,Become better in my IC role,NaN,"B2B Saas, Web3",Series B,Eastern Time (GMT -4),...,0,0,0,0,0,0,0,0,1,0
239,240.0,NaN,Product,Manager of ICs,0-2,Become better in my management role,NaN,"Fintech, AI",Series C,Eastern Time (GMT -4),...,0,0,0,0,0,0,0,0,0,0


In [88]:
mlb_df = pd.DataFrame(
    mlb.fit_transform(t),
    columns = mlb.classes_,
    index = test.index)
print(mlb_df.columns.values)

['AI' 'Agency' 'AgriTech' 'Agritech' 'B2B Saas' 'Biotech' 'Climate Tech'
 'Community' 'Construction Tech' 'Consumer' 'Cybersecurity' 'DTC'
 'DevOps / Dev tools' 'E-commerce' 'EdTech' 'Fintech' 'Food'
 'Future of work' 'Gaming' 'HR' 'Health & Wellness' 'Health Tech' 'IaaS'
 'IaaS/Cloud computing' 'Identity Access Management'
 'Internal enterprise tools' 'IoT' 'Legal Tech' 'MarTech' 'Marketplace'
 'Media' 'Medtech selling to insurance with d2c marketing' 'Non-profit'
 'PropTech' 'Proptech' 'Restaurant Technology' 'Smart Home' 'Supply Chain'
 'Travel' 'Web3' 'igaming affiliation']
